In [7]:
!source mp_env/bin/activate

In [8]:
!pip3 install ultralytics -quiet # object detection model
!pip3 install opencv-python-headless -quiet # for image processing
!pip3 install torch torchvision torchaudio -quiet # for deeplearning


[optparse.groups]Usage:[/]   
  pip3 install \[options] <requirement specifier> \[package-index-options] ...
  pip3 install \[options] -r <requirements file> \[package-index-options] ...
  pip3 install \[options] [-e] <vcs project url> ...
  pip3 install \[options] [-e] <local project path> ...
  pip3 install \[options] <archive url/path> ...

no such option: -u

[optparse.groups]Usage:[/]   
  pip3 install \[options] <requirement specifier> \[package-index-options] ...
  pip3 install \[options] -r <requirements file> \[package-index-options] ...
  pip3 install \[options] [-e] <vcs project url> ...
  pip3 install \[options] [-e] <local project path> ...
  pip3 install \[options] <archive url/path> ...

no such option: -u

[optparse.groups]Usage:[/]   
  pip3 install \[options] <requirement specifier> \[package-index-options] ...
  pip3 install \[options] -r <requirements file> \[package-index-options] ...
  pip3 install \[options] [-e] <vcs project url> ...
  pip3 install \[options] [

In [17]:
!yolo detect train \
  data=Data/Soccernet_yolo/data.yaml \
  model=yolov8n.pt \
  epochs=5 \
  imgsz=640 \
  batch=16


/Users/sandalisingh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics 8.4.14 🚀 Python-3.9.6 torch-2.8.0 CPU (Apple M1)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=Data/Soccernet_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=Non

In [18]:
!yolo detect predict \
  model=runs/detect/train/weights/best.pt \
  source=Data/Soccernet_yolo/test/images/171.png \
  imgsz=640


Ultralytics 8.4.14 🚀 Python-3.9.6 torch-2.8.0 CPU (Apple M1)
Model summary (fused): 73 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /Users/sandalisingh/Documents/SheInnovates/Data/Soccernet_yolo/test/images/171.png: 384x640 9 Player team left (0)s, 10 Player team right (1)s, 36.7ms
Speed: 2.9ms preprocess, 36.7ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /Users/sandalisingh/Documents/SheInnovates/runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [21]:
!yolo detect predict \
  model=runs/detect/train/weights/best.pt \
  source=Data/raw_videos/Video1.mp4 \
  save=True


Ultralytics 8.4.14 🚀 Python-3.9.6 torch-2.8.0 CPU (Apple M1)
Model summary (fused): 73 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (frame 1/750) /Users/sandalisingh/Documents/SheInnovates/Data/raw_videos/Video1.mp4: 384x640 12 Player team left (0)s, 11 Player team right (1)s, 36.9ms
video 1/1 (frame 2/750) /Users/sandalisingh/Documents/SheInnovates/Data/raw_videos/Video1.mp4: 384x640 12 Player team left (0)s, 10 Player team right (1)s, 34.8ms
video 1/1 (frame 3/750) /Users/sandalisingh/Documents/SheInnovates/Data/raw_videos/Video1.mp4: 384x640 11 Player team left (0)s, 11 Player team right (1)s, 38.4ms
video 1/1 (frame 4/750) /Users/sandalisingh/Documents/SheInnovates/Data/raw_videos/Video1.mp4: 384x640 13 Player team left (0)s, 10 Player team right (1)s, 36.0ms
video 1/1 (frame 5/750) /Users/sandalisingh/Documents/SheInnovates/Data/raw_videos/Video1.mp4: 384x640 10 Player team left (0)s, 14 Player team right (1)s, 37.5ms
video 1/1 (frame 6/750) /Users/sandalisingh